# Tiền xử lý dữ liệu

## Tại sao cần phải tiền xử lý dữ liệu
Tiền xử lý dữ liệu là một phần quan trọng trong quy trình phân tích dữ liệu và xây dựng mô hình máy học, giúp cải thiện chất lượng và hiệu suất của mô hình cuối cùng.

## Mục tiêu
Ở phần này, dữ liệu chúng ta thu được là dữ liệu dạng **văn bản**. Tuy nhiên các mô hình học máy chỉ có thể "học" khi dữ liệu là dạng **số thực** nên trong phần này, việc quan trọng nhất đó chính là *Làm thế nào để chuyển dữ liệu sang dạng số thực?* 

## Import các thư viện cần dùng

In [1]:
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import plotly.express as px
from pyvi import ViTokenizer
import matplotlib.pyplot as plt
from collections import Counter

### Đọc dữ liệu từ thư mục data/
Vì dữ liệu được lưu dưới dạng JSON, nên chúng ta có thể sử dụng trực tiếp phương thức `read_json()` trong thư viện `pandas`

In [2]:
data = pd.read_json("../data/dantri.json")
data.head()

,title,category,content
0,"Thi hành án dứt điểm những vụ lớn, phức tạp li...",Xã hội,"Ông Nguyễn Văn Sơn, Phó Tổng cục trưởng Tổng c..."
1,"Món quà của Thủ tướng dành cho những người ""kh...",Xã hội,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph..."
2,Thủ tướng thị sát đường sắt Nhổn - ga Hà Nội: ...,Xã hội,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph..."
3,Đại sứ Saudi Arabia: Mong muốn Việt Nam sớm vư...,Xã hội,Thủ tướng Phạm Minh Chính tiếp Đại sứ Saudi Ar...
4,"Tổng Thanh tra Chính phủ nói về giải pháp ""trị...",Xã hội,Tổng Thanh tra Chính phủ Đoàn Hồng Phong vừa c...


In [3]:
print(f"Dữ liệu gồm {data.shape[0]} theo dõi và {data.shape[1]} thuộc tính")

Dữ liệu gồm 13931 theo dõi và 3 thuộc tính


## Tiền xử lý dữ liệu

### Loại trừ các dữ liệu NaN

In [4]:
data.isnull().sum()

title       0
category    0
content     0
dtype: int64

Dữ liệu trên, không có theo dõi nào có thuộc tính bị NaN hoặc rỗng, do đó, ở phần này chúng ta không cần xử lý phần việc loại trừ các giá trị bị rỗng

> **Lưu ý**: Mục tiêu của bài toán là dựa vào nội dung (**content**) để phân loại dữ liệu thuộc hạng mục nào (**category**). Do đó, có hai lí do để chúng ta chỉ chọn cột **category** và **content**. Thứ nhất, là do giá trị cột **title** thường quá ngắn để có thể phân loại và thứ hai, chúng ta sẽ dựa vào nội dung (**content**) để phân loại.

In [5]:
data = data[["content", "category"]]
data.head()

,content,category
0,"Ông Nguyễn Văn Sơn, Phó Tổng cục trưởng Tổng c...",Xã hội
1,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph...",Xã hội
2,"Sáng 12/2 (mùng 3 Tết Giáp Thìn), Thủ tướng Ph...",Xã hội
3,Thủ tướng Phạm Minh Chính tiếp Đại sứ Saudi Ar...,Xã hội
4,Tổng Thanh tra Chính phủ Đoàn Hồng Phong vừa c...,Xã hội


In [6]:
count = data["category"].value_counts()
num_of_post_each_topic = pd.DataFrame({
    'category': count.index,
    'count': count.values
})

In [7]:
count.index

Index(['Thể thao', 'Giáo dục', 'Sức khỏe', 'Bất động sản', 'Kinh doanh',
       'Văn hóa', 'Xã hội', 'Xe ++', 'Sức mạnh số', 'Thế giới', 'Việc làm',
       'Giải trí', 'Nhân ái', 'An sinh', 'Pháp luật'],
      dtype='object', name='category')

In [8]:
num_of_post_each_topic

,category,count
0,Thể thao,1497
1,Giáo dục,1414
2,Sức khỏe,1270
3,Bất động sản,1229
4,Kinh doanh,1102
5,Văn hóa,1044
6,Xã hội,967
7,Xe ++,910
8,Sức mạnh số,800
9,Thế giới,759


In [9]:
fig = px.histogram(
    num_of_post_each_topic,
    x='category',
    y='count',
    color='category',
    title="Số lượng bài báo trên từng hạng mục",
    color_discrete_sequence=px.colors.qualitative.Set1,
)
fig.update_yaxes(showgrid=True)  # the y-axis is in dollars
fig.update_layout(  # customize font and legend orientation & position
    font_family="Arial",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
    ),
    yaxis_title="Số lượng bài báo",
    xaxis_title="Hạng mục",
)

fig.add_shape(  # add a horizontal "target" line
    type="line",
    line_color="#16FF32",
    line_width=3,
    opacity=1,
    line_dash="dot",
    x0=0,
    x1=1,
    xref="paper",
    y0=929,
    y1=929,
    yref="y",
)

fig.add_annotation(  # add a text callout with arrow
    text="Số lượng ít nhất", x="Pháp luật", y=500, arrowhead=1, showarrow=True
)


fig.show()

e:\PKA-Project\DS-DM-Project\.venv\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


Dựa vào hình trên, nhận thấy rằng, số lượng bài báo thu được ở hạng mục `Thể thao` là nhiều nhất với gần 1k5 bài, trong khi đó, số lượng bài báo thu được ở hạng mục `Pháp luật` là ít nhất với khoảng 400 bài

> Nhìn chung, dữ liệu thu được khoảng 14k bài báo, chưa đủ nhiều đến mức đánh giá bài toán là **Imbalanced Data** nhưng chúng ta có thể xét vào bài toán trên và đánh giá nếu hiệu suất ổn hơn

### Building Vocabulary and Cleaning Text

In [10]:
# Initialize Counter instance
vocab = Counter()

# Get punctuation list
punc = string.punctuation
# Get digits list
digits = string.digits

# Read vietnamese-stopwords.txt in data/
with open("../data/vietnamese-stopwords.txt", encoding='UTF-8') as f:
    lines = [line.rstrip("\n").strip() for line in f]

stopwords = [line.replace(' ', '_') for line in lines]
stopwords[:5]

['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy']

In [11]:
punc_custom = punc.translate(str.maketrans("", "", "_"))

In [12]:
def clean_text(content: str):
    # lowercase content
    content = content.lower()
    # remove white space
    content = " ".join([x for x in content.split()])
    # remove punctuation and digits
    content = content.translate(str.maketrans("", "", punc_custom)).translate(
        str.maketrans("", "", digits)
    )
    # tokenize content
    content = ViTokenizer.tokenize(content)
    # remove stopwords
    tokens = nltk.word_tokenize(content)
    tokens = [token for token in tokens if token not in stopwords]
    # join tokens again
    return " ".join(tokens)


def build_vocab(sentence: str, vocab: Counter):
    tokens = nltk.word_tokenize(sentence)
    vocab.update(tokens)
    
def filter_by_vocad(content: str, vocab_list: list[str]):
    tokens = nltk.word_tokenize(content)
    tokens = [token for token in tokens if token in vocab_list]
    return " ".join(tokens)

In [13]:
data['content'] = data['content'].apply(clean_text)

In [14]:
data.head()

,content,category
0,nguyễn văn sơn phó_tổng_cục_trưởng tổng_cục th...,Xã hội
1,mùng tết giáp thìn thủ_tướng phạm minh đi thăm...,Xã hội
2,mùng tết giáp thìn thủ_tướng phạm minh đi kiểm...,Xã hội
3,thủ_tướng phạm minh tiếp đại_sứ saudi arabia v...,Xã hội
4,tổng_thanh_tra chính_phủ đoàn hồng phong văn_b...,Xã hội


In [15]:
for sentence in tqdm(data['content']):
    build_vocab(sentence, vocab)
print("Successfully Build Vocabulary")

100%|██████████| 13931/13931 [00:11<00:00, 1264.24it/s]

Successfully Build Vocabulary


In [16]:
MIN_OCCURRENCE = 5
vocab_geq_min_occurrence = [k for k, c in vocab.items() if c >= MIN_OCCURRENCE]
print(
    f"Original Vocabulary has {len(vocab.items())} words, after filter by MIN_OCCURRENCE, the result has {len(vocab_geq_min_occurrence)} words"
)

Original Vocabulary has 70398 words, after filter by MIN_OCCURRENCE, the result has 26003 words


In [17]:
with open('../data/vocab.txt', 'w', encoding="UTF-8") as f:
    vocab_data = '\n'.join(vocab_geq_min_occurrence)
    f.write(vocab_data)
    f.close()

In [18]:
with open("../data/vocab.txt", 'r', encoding='UTF-8') as f:
    vocab_saved = [line.rstrip("\n").strip() for line in f]
vocab_saved[:5]

['nguyễn', 'văn', 'sơn', 'phó_tổng_cục_trưởng', 'tổng_cục']

In [19]:
data['content'] = data['content'].apply(lambda x: filter_by_vocad(x, vocab_saved))

In [21]:
data.to_csv('../data/dantri_cleaned.csv', index=False)